In [1]:
import os
import pickle
from aips.data_loaders.outdoors import load_dataframe
from aips import get_engine
from sentence_transformers import SentenceTransformer, SimilarityFunction
from sentence_transformers.quantization import quantize_embeddings
from sentence_transformers.util import cos_sim
from pyspark.sql import SparkSession
from aips import get_engine
import time
import numpy
import math

engine = get_engine()
spark = SparkSession.builder.appName("AIPS").getOrCreate()

#https://github.com/facebookresearch/faiss/wiki/Pre--and-post-processing
#https://github.com/facebookresearch/faiss/wiki

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
![ ! -d 'outdoors' ] && git clone --depth=1 https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../data/outdoors/' && tar -xvf outdoors.tgz -C '../data/outdoors/'

#outdoors_collection = engine.create_collection("outdoors")

Already up to date.
README.md
concepts.pickle
._guesses.csv
guesses.csv
._guesses_all.json
guesses_all.json
outdoors_concepts.pickle
outdoors_embeddings.pickle
._outdoors_golden_answers.csv
outdoors_golden_answers.csv
._outdoors_golden_answers.xlsx
outdoors_golden_answers.xlsx
._outdoors_golden_answers_20210130.csv
outdoors_golden_answers_20210130.csv
outdoors_labels.pickle
outdoors_question_answering_contexts.json
outdoors_questionanswering_test_set.json
outdoors_questionanswering_train_set.json
._posts.csv
posts.csv
predicates.pickle
pull_aips_dependency.py
._question-answer-seed-contexts.csv
question-answer-seed-contexts.csv
question-answer-squad2-guesses.csv
._roberta-base-squad2-outdoors
roberta-base-squad2-outdoors/
roberta-base-squad2-outdoors/._tokenizer_config.json
roberta-base-squad2-outdoors/tokenizer_config.json
roberta-base-squad2-outdoors/._special_tokens_map.json
roberta-base-squad2-outdoors/special_tokens_map.json
roberta-base-squad2-outdoors/._config.json
roberta-base-

In [3]:
dimensions = 1024
#model = SentenceTransformer("tomaarsen/mpnet-base-nli-matryoshka", matryoshka_dims=64)
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1",
                            similarity_fn_name=SimilarityFunction.DOT_PRODUCT,
                            truncate_dim=dimensions)
#_ = model.half()
def get_embeddings(texts, cache_name, ignore_cache=False):
    cache_file_name = f"data/embeddings/{cache_name}.pickle"
    if ignore_cache or not os.path.isfile(cache_file_name):        
        #pool = model.start_multi_process_pool()
        embeddings = model.encode(texts, normalize_embeddings=True)
        #model.stop_multi_process_pool(pool)
        #embeddings = model.encode(texts, convert_to_tensor=False).tolist()
        os.makedirs(os.path.dirname(cache_file_name), exist_ok=True)
        with open(cache_file_name, "wb") as fd:
            pickle.dump(embeddings, fd)
    else:
        with open(cache_file_name, "rb") as fd:
            embeddings = pickle.load(fd)
    return embeddings

## Listing 13.21
### Generating embeddings and benchmark data

In [4]:
import faiss
from aips.data_loaders.outdoors import load_dataframe

def index_embeddings(embeddings, name):
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, name)
    return index

def calculate_outdoors_embeddings():
    outdoors_dataframe = load_dataframe("data/outdoors/posts.csv")
    post_texts = [post["title"] + " " + post["body"]
                  for post in outdoors_dataframe.collect()]
    embeddings = get_embeddings(post_texts, "outdoors_mrl_normed")
    outdoors_data = list(outdoors_dataframe.rdd.map(lambda r: r.asDict()).collect())
    return numpy.array(embeddings), outdoors_data

In [5]:
embeddings, outdoors_dataframe = calculate_outdoors_embeddings()

## Listing 13.22
### int8 quantization

In [6]:
def index_int_embeddings(embeddings, name):
    embeddings = quantize_embeddings(embeddings, precision="int8")
    index = faiss.IndexScalarQuantizer(embeddings.shape[1], faiss.ScalarQuantizer.QT_8bit)
    index.train(embeddings)
    index.add(embeddings)
    faiss.write_index(index, name)
    return index

def execute_search(index, index_name, embedded_query, k=25, log=False,
                   target_time_taken=None, target_size=None):
    start_time = time.time()
    faiss_scores, faiss_doc_ids = index.search(embedded_query, k) 
    time_taken = ((time.time() - start_time) * 1000)
    size = os.path.getsize(index_name)
    improvement_ms, improvement_size = calculate_stat_messages(target_time_taken, time_taken,
                                                               target_size, size)
    print(f"{index_name} search took: {time_taken:.3f} ms {improvement_ms}")
    print(f"{index_name} index size: " + '{:,}'.format(size / 1000000) + f" bytes {improvement_size}")
    if log:
        display(f"Execute search scores: {faiss_scores}")
        display(f"Search ids: {faiss_doc_ids}")
    return faiss_scores, faiss_doc_ids, time_taken, size

def display_results(scores, ids, data):
    results = generate_search_results(scores, ids, data)
    display(results)
    return results

def generate_search_results(faiss_scores, faiss_ids, data):
    scores = list(list(faiss_scores)[0])
    ids = list(faiss_ids)
    results = []
    for i, id in enumerate(ids[0]):
        id = int(id)
        result = {"score": scores[i],
                  "title": data[id]["title"],
                  "body": data[id]["body"],
                  "id": id}
        results.append(result)
    return results

def calculate_recall(scored_full_results, scored_quantized_results):
    full_ids = [r["id"] for r in scored_full_results]
    quantized_ids = [r["id"] for r in scored_quantized_results]
    recall =  (len(set(full_ids).intersection(set(quantized_ids))) /
               len(set(quantized_ids)))
    print("Recall: " + str(recall))

def execute_full_search(embeddings, query_embeddings, outdoors_data):    
    embeddings, outdoors_data = calculate_outdoors_embeddings()
    full_index = index_embeddings(embeddings, "full_out_embs")
    full_search_scores, ids, time, size = execute_search(full_index, "full_out_embs", query_embeddings)
    scored_full_results = generate_search_results(full_search_scores, ids, outdoors_data)
    #display(scored_full_results)
    return scored_full_results, time, size

def calculate_stat_messages(target_time, time_taken, target_size, size):
    if target_time:
        t = round((target_time - time_taken) * 100 / target_time, 2)
    improvement_ms = f"({t}% improvement)" if target_time else ""
    improvement_size = f"({round((target_size - size) * 100 / target_size, 2)}% improvement)" if target_size else ""
    return improvement_ms, improvement_size

embeddings, outdoors_data = calculate_outdoors_embeddings()
query_embeddings = model.encode(["tent poles"], convert_to_numpy=True, normalize_embeddings=True)
scored_full_results, target_time, target_size = \
    execute_full_search(embeddings, query_embeddings, outdoors_data)

int8_index = index_int_embeddings(embeddings, "int8_out_embs")
int8_query_embeddings = quantize_embeddings(query_embeddings, calibration_embeddings=embeddings,
                                            precision="int8")
int8_search_scores, int8_ids, _, _ = execute_search(int8_index, "int8_out_embs", int8_query_embeddings,
                                              target_time_taken=target_time,
                                              target_size=target_size)
scored_int8_results = generate_search_results(int8_search_scores, int8_ids, outdoors_data)

calculate_recall(scored_full_results, scored_int8_results)

full_out_embs search took: 3.871 ms 
full_out_embs index size: 75.595821 bytes 
int8_out_embs search took: 4.059 ms (-4.87% improvement)
int8_out_embs index size: 18.907217 bytes (74.99% improvement)
Recall: 0.96


## Listing 13.23
### Binary Quantization

In [7]:
def binary_quantize_embeddings(embeddings):
    quantized_embs = numpy.where(embeddings < 0, 0, 1).astype(numpy.uint8) # binarize
    return numpy.packbits(quantized_embs).reshape(quantized_embs.shape[0], -1)
    
def index_binary_embeddings(embeddings):
    #binary_embeddings = binary_quantize_embeddings(embeddings)
    binary_embeddings = numpy.packbits(embeddings > 0).reshape(embeddings.shape[0], -1)
    print("Binary embeddings computed. Shape:", binary_embeddings.shape)
    index = faiss.IndexBinaryFlat(binary_embeddings.shape[1] * 8)
    index.add(binary_embeddings)
    faiss.write_index_binary(index, "binary_out_embs")
    return index

binary_index = index_binary_embeddings(embeddings)
binary_query_embeddings = binary_quantize_embeddings(query_embeddings)
binary_search_scores, binary_ids, _, _ = execute_search(binary_index, "binary_out_embs",
                                                  binary_query_embeddings, log=False,
                                                  target_time_taken=target_time,
                                                  target_size=target_size)
scored_binary_results = generate_search_results(binary_search_scores, binary_ids, outdoors_data)

calculate_recall(scored_full_results, scored_binary_results)
# (vs. 5.000 ms, 90% reduction)

Binary embeddings computed. Shape: (18456, 128)
binary_out_embs search took: 4.330 ms (-11.87% improvement)
binary_out_embs index size: 2.362401 bytes (96.87% improvement)
Recall: 0.68


## Listing 13.24
### Matroyoshka Learned Representations

In [10]:
def evaluate_mrl_quantized_searches(query):
    embeddings, outdoors_data = calculate_outdoors_embeddings()
    query_embeddings = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    for slice in [512, 256, 128]:
        scaled_embeddings = numpy.array(list(map(lambda e: e[:slice], embeddings)))
        index = index_embeddings(scaled_embeddings, f"mrl_out_embs_{slice}")
        scaled_query_embeddings = numpy.array(list(map(lambda qe: qe[:slice], query_embeddings)))
        search_scores, ids, _, _ = execute_search(index, f"mrl_out_embs_{slice}", scaled_query_embeddings,
                                            target_time_taken=target_time,
                                            target_size=target_size)
        scored_results = generate_search_results(search_scores, ids, outdoors_data)
        calculate_recall(scored_full_results, scored_results)
        #yield calculate_recall

evaluate_mrl_quantized_searches("hiking trails")

#full_out_embs search took: 3.900 ms
#full_out_embs index size: 75,595,821 bytes

mrl_out_embs_512 search took: 2.366 ms (38.89% improvement)
mrl_out_embs_512 index size: 37.797933 bytes (50.0% improvement)
Recall: 0.0
mrl_out_embs_256 search took: 1.102 ms (71.52% improvement)
mrl_out_embs_256 index size: 18.898989 bytes (75.0% improvement)
Recall: 0.0
mrl_out_embs_128 search took: 0.509 ms (86.86% improvement)
mrl_out_embs_128 index size: 9.449517 bytes (87.5% improvement)
Recall: 0.0


## Listing 13.25
### Product quantizationQ

In [15]:
def index_pq_embeddings(embeddings, name="pq_out_embs"):    
    dimensions = embeddings.shape[1]
    sub_vectors = 8
    subquantizer_bits = 8
    #faiss::IndexIVFPQ, IndexIVFPQR
    index = faiss.IndexPQ(dimensions, sub_vectors, subquantizer_bits)
    index.train(embeddings)
    index.add(embeddings)
    faiss.write_index(index, name)
    return index

def index_pq_reranked_embeddings(embeddings, name="pq_out_embs"):    
    dimensions = embeddings.shape[1]
    sub_vectors = 8
    subquantizer_bits = 8
    #faiss::IndexIVFPQ, IndexIVFPQR
    index = faiss.IndexPQ(dimensions, sub_vectors, subquantizer_bits)
    rereanking_pq_index = faiss.IndexRefineFlat(index)
    rereanking_pq_index.train(embeddings)
    rereanking_pq_index.add(embeddings)
    faiss.write_index(rereanking_pq_index, name)
    return index

embeddings, outdoors_data = calculate_outdoors_embeddings()
index = index_pq_reranked_embeddings(embeddings)
scores, ids, _, _ = execute_search(index, "pq_out_embs", query_embeddings,
                                   target_time_taken=target_time,
                                   target_size=target_size)
scored_results = generate_search_results(scores, ids, outdoors_data)
calculate_recall(scored_full_results, scored_results)

## Listing 13.26
### Quantization and reranking